In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import scipy as scipy
from toolbar.significance_test import r_test

In [2]:
K_type = xr.open_dataset(r"D:/PyFile/p2/data/Time_type_AverFiltAll0.9%_0.3%_3.nc")

In [5]:
print('90%', r_test(62, 0.10))
print('95%', r_test(62, 0.05))
print('99%', r_test(62, 0.01))

90% 0.21083186294286696
95% 0.2500349005300471
99% 0.3248184473571816


In [7]:
type_1 = K_type.sel(type=1)['K'][:-1]
type_2 = K_type.sel(type=2)['K'][:-1]
type_3 = K_type.sel(type=3)['K'][:-1]

# 计算相关系数
corr_123 = np.corrcoef([type_1, type_2, type_3])
print(corr_123)

[[ 1.         -0.28453486 -0.16799297]
 [-0.28453486  1.          0.05889402]
 [-0.16799297  0.05889402  1.        ]]


In [3]:
# 中下游主导型
type_1 = K_type.sel(type=1)['K']
index_1 = np.load(r"D:\PyFile\p2\data\type1_P_warm.npy")
index_2 = np.load(r"D:\PyFile\p2\data\type1_LN.npy")

corr = np.corrcoef([type_1, index_1, index_2])
print(corr)

# 计算偏相关
corr_type_i1 = (corr[0,1] - corr[0,2] * corr[1,2]) / np.sqrt((1 - corr[0,2]**2) * (1 - corr[1,2]**2))
print('因子1偏相关:',corr_type_i1)
corr_type_i2 = (corr[0,2] - corr[0,1] * corr[1,2]) / np.sqrt((1 - corr[0,1]**2) * (1 - corr[1,2]**2))
print('因子2偏相关:',corr_type_i2)

# 新增回归分析代码
import statsmodels.api as sm

# 构建回归矩阵 (n samples × 2 features)
X = np.column_stack((index_1, index_2))
X = sm.add_constant(X)  # 添加常数项
y = type_1.values

# 执行回归
model = sm.OLS(y, X).fit()

# 输出回归结果
print("\n二元线性回归方程：")
print(f"Y = {model.params[0]:.3f} + {model.params[1]:.3f}*X1 + {model.params[2]:.3f}*X2")
print("\n回归结果摘要：")
print(model.summary())

# 计算标准化回归系数（Beta系数）
beta_coef = model.params[1:] * np.std(X[:,1:], axis=0) / np.std(y)
print("\n标准化回归系数：")
print(f"Beta1(X1): {beta_coef[0]:.3f}, Beta2(X2): {beta_coef[1]:.3f}")

# 在回归结果后添加：
print("\n方差分解:")
print(f"总解释方差 (R²): {model.rsquared:.2%}")
print(f"因子1独特贡献: {beta_coef[0]**2:.2%}")
print(f"因子2独特贡献: {beta_coef[1]**2:.2%}")

[[1.         0.37154693 0.37998367]
 [0.37154693 1.         0.52493652]
 [0.37998367 0.52493652 1.        ]]
因子1偏相关: 0.21856932670185988
因子2偏相关: 0.23404501849745252

二元线性回归方程：
Y = 18.500 + 1.806*X1 + 1.941*X2

回归结果摘要：
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     6.708
Date:                Sun, 05 Oct 2025   Prob (F-statistic):            0.00237
Time:                        15:54:07   Log-Likelihood:                -207.39
No. Observations:                  62   AIC:                             420.8
Df Residuals:                      59   BIC:                             427.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                  

In [9]:
# 全域一致型
type_2 = K_type.sel(type=2)['K'][:-1]
index_3 = np.load(r"D:\PyFile\p2\data\type2_In_pre.npy")
index_4 = np.load(r"D:\PyFile\p2\data\type2_LN.npy")

corr = np.corrcoef([type_2, index_3, index_4])
print(corr)

# 计算偏相关
corr_type_i3 = (corr[0,1] - corr[0,2] * corr[1,2]) / np.sqrt((1 - corr[0,2]**2) * (1 - corr[1,2]**2))
print('因子1偏相关:',corr_type_i3)
corr_type_i4 = (corr[0,2] - corr[0,1] * corr[1,2]) / np.sqrt((1 - corr[0,1]**2) * (1 - corr[1,2]**2))
print('因子2偏相关:',corr_type_i4)

# 新增回归分析代码
import statsmodels.api as sm

# 构建回归矩阵 (n samples × 2 features)
X = np.column_stack((index_3, index_4))
X = sm.add_constant(X)  # 添加常数项
y = type_2.values

# 执行回归
model = sm.OLS(y, X).fit()

# 输出回归结果
print("\n二元线性回归方程：")
print(f"Y = {model.params[0]:.3f} + {model.params[1]:.3f}*X1 + {model.params[2]:.3f}*X2")
print("\n回归结果摘要：")
print(model.summary())

# 计算标准化回归系数（Beta系数）
beta_coef = model.params[1:] * np.std(X[:,1:], axis=0) / np.std(y)
print("\n标准化回归系数：")
print(f"Beta1(X1): {beta_coef[0]:.3f}, Beta2(X2): {beta_coef[1]:.3f}")

# 在回归结果后添加：
print("\n方差分解:")
print(f"总解释方差 (R²): {model.rsquared:.2%}")
print(f"因子1独特贡献: {beta_coef[0]**2:.2%}")
print(f"因子2独特贡献: {beta_coef[1]**2:.2%}")

[[1.         0.32633783 0.33543504]
 [0.32633783 1.         0.11907316]
 [0.33543504 0.11907316 1.        ]]
因子1偏相关: 0.30618822248498007
因子2偏相关: 0.3160021894840745

二元线性回归方程：
Y = 12.410 + 2.425*X1 + 2.511*X2

回归结果摘要：
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     7.057
Date:                Mon, 29 Sep 2025   Prob (F-statistic):            0.00181
Time:                        15:16:00   Log-Likelihood:                -209.35
No. Observations:                  61   AIC:                             424.7
Df Residuals:                      58   BIC:                             431.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                   

In [10]:
# 上游主导型
type_3 = K_type.sel(type=3)['K']
index_5 = np.load(r"D:\PyFile\p2\data\type3_EL.npy")
index_6 = np.load(r"D:\PyFile\p2\data\type3_P_cold.npy")

corr = np.corrcoef([type_3, index_5, index_6])
print(corr)
# 计算偏相关
corr_type_i5 = (corr[0,1] - corr[0,2] * corr[1,2]) / np.sqrt((1 - corr[0,2]**2) * (1 - corr[1,2]**2))
print('因子1偏相关:',corr_type_i5)
corr_type_i6 = (corr[0,2] - corr[0,1] * corr[1,2]) / np.sqrt((1 - corr[0,1]**2) * (1 - corr[1,2]**2))
print('因子2偏相关:',corr_type_i6)


# 新增回归分析代码
import statsmodels.api as sm

# 构建回归矩阵 (n samples × 2 features)
X = np.column_stack((index_5, index_6))
X = sm.add_constant(X)  # 添加常数项
y = type_3.values

# 执行回归
model = sm.OLS(y, X).fit()

# 输出回归结果
print("\n二元线性回归方程：")
print(f"Y = {model.params[0]:.3f} + {model.params[1]:.3f}*X1 + {model.params[2]:.3f}*X2")
print("\n回归结果摘要：")
print(model.summary())

# 计算标准化回归系数（Beta系数）
beta_coef = model.params[1:] * np.std(X[:,1:], axis=0) / np.std(y)
print("\n标准化回归系数：")
print(f"Beta1(X1): {beta_coef[0]:.3f}, Beta2(X2): {beta_coef[1]:.3f}")

# 在回归结果后添加：
print("\n方差分解:")
print(f"总解释方差 (R²): {model.rsquared:.2%}")
print(f"因子1独特贡献: {beta_coef[0]**2:.2%}")
print(f"因子2独特贡献: {beta_coef[1]**2:.2%}")

[[ 1.          0.23725679  0.21681406]
 [ 0.23725679  1.         -0.1201412 ]
 [ 0.21681406 -0.1201412   1.        ]]
因子1偏相关: 0.27168885548293165
因子2偏相关: 0.25437129244142537

二元线性回归方程：
Y = 4.177 + 1.283*X1 + 1.195*X2

回归结果摘要：
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     3.922
Date:                Mon, 29 Sep 2025   Prob (F-statistic):             0.0252
Time:                        15:16:20   Log-Likelihood:                -181.37
No. Observations:                  62   AIC:                             368.7
Df Residuals:                      59   BIC:                             375.1
Df Model:                           2                                         
Covariance Type:            nonrobust                          